# Orchestrator-Workers Workflow
# In this workflow, a central LLM dynamically breaks down tasks, delegates them to worker LLMs, and synthesizes their results.

# When to use this workflow
# This workflow is well-suited for complex tasks where you can't predict the subtasks needed. The key difference from simple parallelization is its flexibility—subtasks aren't pre-defined, but determined by the orchestrator based on the specific input.

In [2]:
%pip install -qU langchain-google-genai

Note: you may need to restart the kernel to use updated packages.


DEPRECATION: uvicorn 0.14.0 has a non-standard dependency specifier click>=7.*. pip 24.1 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of uvicorn or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063

[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import os
from dotenv import load_dotenv

load_dotenv()

os.environ["GOOGLE_API_KEY"] = os.getenv("GEMINI_API_KEY")

from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash-latest")



# Let's switch to python now for creating better systems.

In [3]:
n=input("Enter the query on which you want to perform the task:")

orchestrator_prompt=f"""
You are an manager of a company who needs to allocate tasks to workers.
You are given a task: {n}
You have 3 workers available.

worker1 is a sentiment analysis expert.
worker2 is a classification expert which can classify the text into technical,product,marketing,sales,finance,etc
worker3 is a summarization expert.

Return the output in the following format in XML format(strictly follow the format, nothing else should be outputed):

<worker1>
Actual Task: (Eg- "Analyze the sentiment of the text")
</worker1>
<worker2>
Actual Task: (Eg- "Classify the text into who should be assigned to this ticket from technical,product,marketing,sales,finance,etc")
</worker2>
<worker3>
Actual Task: (Eg- "Summarize the text")
</worker3>
"""

def extract_worker1_task(text):
    return text.split("worker1")[1].split("</worker1>")[0].strip()

def extract_worker2_task(text):
    return text.split("worker2")[1].split("</worker2>")[0].strip()

def extract_worker3_task(text):
    return text.split("worker3")[1].split("</worker3>")[0].strip()

result=llm.invoke(orchestrator_prompt)

In [15]:
print(result.content.split('```')[1].split('<worker1>')[1].split("/worker1")[0].split("<")[0].strip())

Actual Task: Analyze the sentiment expressed in the bill-related text ("my bill is very high"). Determine if the sentiment is negative, positive, or neutral.
